In [13]:
# cell 1
import numpy as np
import math
from scipy.stats import ortho_group # for generate input data 
import time 
import random 
debug = False

## Cholesky factorization

Funtion to calculate Cholesky Factorization
Given the input:\
$\;\;\;\;\;\;$ 1. Matrix X is symmetric and square.\
Return:\
$\;\;\;\;\;\;$ 1. Lower triangular matrix R has the same size with the input X

In [14]:
# cell 2
def chol(X):
    n, m = X.shape
    R = np.zeros(n*n)
    if m != n: 
        print('Matrix to process Chol is not square')
        return np.zeros([n, n])
    
    for i in range(n):
        r = i*n
        for j in range(i):
            c = j*n
            R[r+j] = (X[i][j] - sum(np.multiply(R[r:r+j], R[c:c+j])))/R[c+j]
            
        tem1 = X[i][i] - sum(np.multiply(R[r:r+i], R[r:r+i]))
        if tem1 < 0:
            print('Cholesky: Matrix is not posdef')
            return np.zeros([n, n])
        R[r+i] = math.sqrt(tem1)
        
    return np.reshape(R, [n,n])

## Solving lower triangular system 

Function to solve lower triangular system R'y = M'b\
Input: \
$\;\;\;\;\;\;$ 1. Lower triangular matrix R' size kxk \
$\;\;\;\;\;\;$ 2. Matrix M' size kxn (or kxm) \
$\;\;\;\;\;\;$ 3. Vector b size nx1 (or mx1) \
Output: \
$\;\;\;\;\;\;$ A vector y size kx1 


In [15]:
# cell 3
def slts(R, Mt, b):
    #calculate M'b
    n, m = R.shape
    p, q = Mt.shape
    g = b.shape[0]
    
    if m != n or n != p or q != g: 
        print('SLTS function: size of matrixes is  not comparitible')
        if debug:
            print(m, n, p, q, g)
        return np.zeros(n)
    
    v = np.matmul(Mt, b)
    
    y = np.zeros(n)
    
    Rtem = np.reshape(R, n*m)
    
    for i in range(n):
        r = i*n
        R1 = Rtem[r:r+i]
        ytem = y[0:i]
        y[i]= (v[i] - sum(np.multiply(R1, ytem)))/R[i][i]
    return y

## Solving upper triangular system 

Function to solve upper triangular system Rx = y \
Input: \
$\;\;\;\;\;\;$ 1. Upper triangular matrix R size kxk \
$\;\;\;\;\;\;$ 2. Vector y size kx1 \
Output: \
$\;\;\;\;\;\;$ Vector x size kx1

In [16]:
# cell 4
def suts (Rt, y):
    n, m = Rt.shape
    g = y.shape[0]
    
    if n != m or n != g:
        print('SUTS function: size of matrixes is  not comparitible')
    
    x = np.zeros(n)
    Rtem = np.reshape(Rt, n*m)
    for i in reversed(range(n)):
        r = i*n
        R1 = Rtem[r+i+1:r+n]
        xtem = x[i+1:n]
        x[i] = (y[i] - sum(np.multiply(R1, xtem)))/Rt[i][i]
    return x

## Finding solution for the subproblem 

In [17]:
# cell 5
def argmin(A, M):
    
    m, n = A.shape # mxn or nxm
    h, k = M.shape # nxk or mxk
    
    if n != h:
        print("ARGMIN function: size of matrixes is not compatible")
    
    N = []
    # calculate M'*M
    Mt = M.transpose() #kxn
    X = np.matmul(Mt, M) #kxk
    
    start = time.time()
    R = chol(X) # find cholesky factor of M'M, R is a lower triangular matrix size kxk
    stop1 = time.time()
    Rt = R.transpose()
    
    
    for i in range(m):
        b = A[i] #1xn
        y = slts(R, Mt, b) #kx1
        x = suts(Rt, y) #kx1
        xt = np.squeeze(x)
        N.append(xt)
    
    stop2 = time.time()
    if debug:
        print("Runtime of chol in argmin: ", stop1 - start)
        print("Time to solve linear system: ", stop2 - stop1)
    N = np.array(N)
    return N

## Main function

Main function of the algorithm. \
Given the input:\
$\;\;\;\;\;\;$ 1. Matrix A is full rank with size mxn.\
$\;\;\;\;\;\;$ 2. The expected rank k \
$\;\;\;\;\;\;$ 3. Maximun number of iteration t \
$\;\;\;\;\;\;$ 4. Threshold ep = ||UV_old - UV|| \
Return:\
$\;\;\;\;\;\;$ 2 matrix U size mxk and V size kxn

In [6]:
#  cell 6
def lrd (A, k, t, ep):
    # get size of matrix A
    m, n = A.shape
    
    if k > min(m, n):
        print("k is greater than rank(A)")
        return
    
    # generate randomly matrix V
    V = np.random.rand(k, n)
    U = np.zeros([m, k])
        
    Vt = V.transpose()
    At = A.transpose()
    
    UVold = np.zeros(A.shape)
    
    avetime = 0
    
    for i in range(t):   
        # iterate to optimize U and V
        start = time.time()
        U = argmin(A, Vt)
        Vt = argmin(At, U)
        V = Vt.transpose()
        
        #check for stop condition
        UV = np.matmul(U, V)
        e = np.linalg.norm(UV - UVold)
        UVold = np.matmul(U, V)
        stop2 = time.time()
        avetime += stop2 - start
        print(i, "\t", e, "\t", np.linalg.norm(A - UV, 2))
        
        if debug:
            print("Time argmin: ", stop1 - start)
            print("Time of the rest: ", stop2 - stop1)
            
        if e <= ep:
            print('LRD: Reach stopping condition')
            break     
    print("Number of iter: ", i)      
    print("Final err: " , e)
    print("Average time for each iter: ", avetime/i)
    return U, V
        

## Generate data

Generate data function. \
Given the input:\
$\;\;\;\;\;\;$ 1. m \
$\;\;\;\;\;\;$ 2. n \
$\;\;\;\;\;\;$ 3. k \
Return:\
$\;\;\;\;\;\;$ A full rank matrix A \
This function also gives us the information of the k-th singular value of matrix A

In [7]:
# cell 7
def gendata(m, n, k):
    if k > min(m, n):
        print("k is greater than rank(A)")
        return

    # using SVD to generate full rank matrix A 
    U_SVD = ortho_group.rvs(dim=m)

    V_SVD = ortho_group.rvs(dim=n)
    V_SVDT = np.transpose(V_SVD)

    sig = []
    for i in range(min(m, n)):
        x = random.random()
        sig.append(x*50)

    sig.sort(reverse=True)


    Sigma = np.zeros([m, n])
    for i in range(len(sig)):
        Sigma[i][i] = sig[i]


    print("k-th singular value of A ", sig[k])

    A = np.matmul(np.matmul(U_SVD, Sigma), V_SVDT)
    return A

In [ ]:
# cell 8 
m = 100
n = 100
k = 10
A = gendata(m, n, k)
U, V = lrd(A, k, 1000, 0.0001)
print("||A-UV||2 = ", np.linalg.norm(A - np.matmul(U, V), 2))

# Testing part

## Test the precision of functions

We calculate norm to verify the precision of our functions 

In [8]:
mt = random.randint(100, 1000)
nt = random.randint(100, 1000)

TM = np.random.rand(mt,nt)
TMt = np.transpose(TM)
TMC = np.matmul(TMt, TM)
print(TM.shape)

start2 = time.time()
Lowerl = np.linalg.cholesky(TMC)
stop2 = time.time()
print("Builtin function: ", stop2 - start2)

# Test Cholesky
start1 = time.time()
Lower = chol(TMC)
stop1 = time.time()
print("Our function chol: ", stop1 - start1)


Upper = np.transpose(Lower) 
LU = np.matmul(Lower, Upper)
D = TMC - LU
print("Error of Cholesky: ", np.linalg.norm(D)/np.linalg.norm(Lower))

# print("Compare error with builin function", np.linalg.norm(Lower - Lowerl))

# Test lts
bt = np.random.rand(mt)

lstart = time.time()
yt = slts(Lower, TMt, bt)
lstop = time.time()

Ll = np.matmul(Lower, yt)
Rl = np.matmul(TMt, bt)
Dl = Ll - Rl
print ("Error of lower triangular system: ", np.linalg.norm(Dl)/np.linalg.norm(yt))
print ("Time of lts: ", lstop - lstart)

# Test uts 
ustart = time.time()
xt = suts(Upper, yt)
ustop = time.time() 

Lu = np.matmul(Upper, xt)
Du = Lu - yt
print("Error of upper triangular system: ", np.linalg.norm(Du)/np.linalg.norm(xt))
print("Running time of uts: ", ustop - ustart)

(480, 375)
Builtin function:  0.017323017120361328
Our function chol:  2.0304698944091797
Error of Cholesky:  2.8701393228849516e-14
Error of lower triangular system:  9.170705556512418e-14
Time of lts:  0.01717209815979004
Error of upper triangular system:  4.900386781992903e-15
Running time of uts:  0.01717662811279297


## Test running time and scale of Cholesky function

In [ ]:
for i in range(10, 1000, 10):
    sumtime = 0
    for j in range(10): 
        TMtime = np.random.rand(i,i)
        TMtimet = np.transpose(TMtime)
        TMCtime = np.matmul(TMtimet, TMtime)

        tstart = time.time()
        chol(TMCtime)
        tstop = time.time()
        sumtime += tstop - tstart
    print(sumtime/10)

# Backup code

## Raw Cholesky (without optimization for running time)

In [ ]:
def chol_backup(X):
    n, m = X.shape
    R = np.zeros([n, n])
    if m != n: 
        print('Matrix to process Chol is not square')
        return np.zeros([n, n])
    for i in range(n):
        for j in range(i): 
            tem = 0
            for t in range(j):
                tem += R[i][t]*R[j][t]
            R[i][j] = (X[i][j] - tem)/R[j][j]
        
        tem = 0
        for t in range(i):
            tem += R[i][t]*R[i][t]

        if X[i][i] - tem < 0:
            print('Cholesky: Matrix is not posdef')
            return np.zeros([n, n])
        R[i][i] = math.sqrt(X[i][i] - tem)
    return R